In [1]:
%cd ..

/Users/koloss/Desktop/CS224N/final_project/RobustQA


In [12]:
from transformers import DistilBertTokenizerFast
import util

In [38]:
import pandas as pd
import numpy as np

In [ ]:

def get_dataset(args, datasets, data_dir, tokenizer, split_name, debug=-1):
    datasets = datasets.split(',')
    dataset_dict = None
    dataset_name = ''
    label = 0
    for dataset in datasets:
        dataset_name += f'_{dataset}'
        dataset_dict_curr = util.read_squad(f'{data_dir}/{dataset}')
        dataset_dict_curr['label'] = label
        dataset_dict = util.merge(dataset_dict, dataset_dict_curr)
        label += 1
    data_encodings = read_and_process(args, tokenizer, dataset_dict, data_dir, dataset_name, split_name)
    return util.QADataset(data_encodings, train=(split_name == 'train')), dataset_dict, label


In [10]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [166]:
datasets = 'duorc,race'
data_dir = 'datasets/oodomain_train'

In [167]:
datasets = datasets.split(',')
dataset_dict = None
dataset_name = ''
label = 0
for dataset in datasets:
    dataset_name += f'_{dataset}'
    dataset_dict_curr = util.read_squad(f'{data_dir}/{dataset}')
    dataset_dict_curr['label'] = label
    dataset_dict = util.merge(dataset_dict, dataset_dict_curr)
    label += 1
#data_encodings = read_and_process(args, tokenizer, dataset_dict, data_dir, dataset_name, split_name)



In [168]:
dataset_dict.keys()

dict_keys(['question', 'context', 'id', 'answer', 'label'])

In [169]:
changes=dict(zip([" game ", " set "], [" lame ", " bet "]))

df = pd.DataFrame({x: dataset_dict[x] for x in dataset_dict if x not in ['label']})
df['start_char'] = df.answer.apply(lambda x : x['answer_start'][0])
df['end_char'] = df['start_char']+ df.answer.apply(lambda x : len(x['text'][0]))
df['final_answer'] = [A[B:C] for A, B, C in zip(df.context, df['start_char'],df['end_char'])]
df['context'] = df.context.str.strip().replace(changes,regex=True)

##df['new_context'] = df.context.str.strip().replace(changes,regex=True)
##df['new_answer'] = [A[B:C] for A, B, C in zip(df['new_context'], df['start_char'],df['end_char'])]
new_dataset_dict = df[[i for i in dataset_dict.keys() if i!= 'label']].to_dict()
new_dataset_dict ['label'] = dataset_dict['label']

In [156]:
#df[[i for i in dataset_dict.keys() if i!= 'label']].to_dict()

In [172]:
df.head()

,question,context,id,answer,start_char,end_char,final_answer
0,What was Jill's mother's face burned by?,"New Orleans, Louisiana, 1927. An enraged posse...",d94a42693350473581ff79dc91c91e04,"{'answer_start': [2476], 'text': ['acid']}",2476,2480,acid
1,What year did a lynch mod muder someone who th...,"New Orleans, Louisiana, 1927. An enraged posse...",f41dbe24bed44870a8ad36c87dda59a2,"{'answer_start': [26], 'text': ['1927']}",26,30,1927
2,Whose corpse is in the morgue?,"New Orleans, Louisiana, 1927. An enraged posse...",34f8093a16b64c8097bdaa03cccdef37,"{'answer_start': [2873], 'text': ['Mary-Anne']}",2873,2882,Mary-Anne
3,What is the name of the blind girl?,"New Orleans, Louisiana, 1927. An enraged posse...",b6a13cac6289435697e8ff98d55854a9,"{'answer_start': [3659], 'text': ['Emily']}",3659,3664,Emily
4,Whose corpse does Liza see?,"New Orleans, Louisiana, 1927. An enraged posse...",52ffe790d24a41669c08a240c1d45114,"{'answer_start': [2348], 'text': ['Jill']}",2348,2352,Jill
